# Method

In [2]:
# dataFrame을 넣고 실행하면 X_data, y_data, vocab_size, max_len를 돌려주는 get_X_y_vacabsize_maxlen method
# @name   : get_X_y_vacabsize_maxlen
# @pram   : DataFrame
# @return : np.array, list, int, int
def get_X_y_vacabsize_maxlen(df):
    X_data = df['comment']
    y_data = df['bad']
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_data)
    sequences = tokenizer.texts_to_sequences(X_data)
    df['comment'] = sequences
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = max(len(l) for l in sequences)
    data = pad_sequences(sequences, maxlen=max_len)
    X_data = (data)
    y_data = df['bad']
    return X_data, y_data, vocab_size, max_len

# vocab_size, max_len를 넣고 실행하면 CNN모델을 만들어주는 CNNmodel method
# @name   : CNNmodel
# @pram   : int, int
# @return : Sequential
def CNNmodel(vocab_size, max_len):
    model = Sequential()
    model.add(Embedding(vocab_size, 32, input_length=max_len))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# vocab_size, max_len를 넣고 실행하면 LSTM모델을 만들어주는 LSTMmodel method
# @name   : LSTMmodel
# @pram   : int, int
# @return : Sequential
def LSTMmodel(vocab_size,max_len):
    model = Sequential()
    model.add(Embedding(vocab_size, 32))
    model.add(LSTM(max_len, activation='tanh'))
    model.add(Dense(units=1))
    model.add(Activation('relu'))
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

# vocab_size를 넣으면 RNN모델을 만들어주는 RNNmodel method
# @name   : RNNmodel
# @pram   : int
# @return : Sequential
def RNNmodel(vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
    model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model

# import

In [3]:
from keras.preprocessing import sequence

from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pyprind
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, SimpleRNN, Embedding, Dense, Activation, Dropout, Flatten

Using TensorFlow backend.


## 처리 방식에 따른 네 가지 데이터 호출

In [5]:
sulliComments_nouns = pd.read_csv('sulliComments_nouns.csv',encoding='cp949')
sulliComments_nounsJaso = pd.read_csv('sulliComments_nounsJaso.csv',encoding='cp949')
sulliComments_morps = pd.read_csv('sulliComments_morps.csv',encoding='cp949')
sulliComments_morpsJaso = pd.read_csv('sulliComments_morpsJaso.csv',encoding='cp949')

## 처리 방식 별 X, y, vocab_size, max_len 지정

In [6]:
nouns_X_data,nouns_y_data, nouns_vocab_size, nouns_max_len = get_X_y_vacabsize_maxlen(sulliComments_nouns)
nounsJaso_X_data,nounsJaso_y_data, nounsJaso_vocab_size, nounsJaso_max_len = get_X_y_vacabsize_maxlen(sulliComments_nounsJaso)
morps_X_data,morps_y_data, morps_vocab_size, morps_max_len = get_X_y_vacabsize_maxlen(sulliComments_morps)
morpsJaso_X_data,morpsJaso_y_data, morpsJaso_vocab_size, morpsJaso_max_len = get_X_y_vacabsize_maxlen(sulliComments_morpsJaso)

## 처리 방식 별 train, valid, test set 생성(80:20rule)

In [7]:
nouns_X_train, nouns_X_test, nouns_y_train, nouns_y_test = train_test_split(nouns_X_data, nouns_y_data, test_size=0.2, random_state=0)
nouns_X_train, nouns_X_valid, nouns_y_train, nouns_y_valid = train_test_split(nouns_X_train, nouns_y_train, test_size=0.2, random_state=0)

nounsJaso_X_train, nounsJaso_X_test, nounsJaso_y_train, nounsJaso_y_test = train_test_split(nounsJaso_X_data, nounsJaso_y_data, test_size=0.2, random_state=0)
nounsJaso_X_train, nounsJaso_X_valid, nounsJaso_y_train, nounsJaso_y_valid = train_test_split(nounsJaso_X_train, nounsJaso_y_train, test_size=0.2, random_state=0)

morps_X_train, morps_X_test, morps_y_train, morps_y_test = train_test_split(morps_X_data, morps_y_data, test_size=0.2, random_state=0)
morps_X_train, morps_X_valid, morps_y_train, morps_y_valid = train_test_split(morps_X_train, morps_y_train, test_size=0.2, random_state=0)

morpsJaso_X_train, morpsJaso_X_test, morpsJaso_y_train, morpsJaso_y_test = train_test_split(morpsJaso_X_data, morpsJaso_y_data, test_size=0.2, random_state=0)
morpsJaso_X_train, morpsJaso_X_valid, morpsJaso_y_train, morpsJaso_y_valid = train_test_split(morpsJaso_X_train, morpsJaso_y_train, test_size=0.2, random_state=0)

## 처리 방식 별 12개의 모델 생성

In [8]:
nouns_RNN_model = RNNmodel(nouns_vocab_size)
nouns_LSTM_model = LSTMmodel(nouns_vocab_size, nouns_max_len)
nouns_CNN_model = CNNmodel(nouns_vocab_size, nouns_max_len)

nounsJaso_RNN_model = RNNmodel(nounsJaso_vocab_size)
nounsJaso_LSTM_model = LSTMmodel(nounsJaso_vocab_size, nounsJaso_max_len)
nounsJaso_CNN_model = CNNmodel(nounsJaso_vocab_size, nounsJaso_max_len)

morps_RNN_model = RNNmodel(morps_vocab_size)
morps_LSTM_model = LSTMmodel(morps_vocab_size, morps_max_len)
morps_CNN_model = CNNmodel(morps_vocab_size, morps_max_len)

morpsJaso_RNN_model = RNNmodel(morpsJaso_vocab_size)
morpsJaso_LSTM_model = LSTMmodel(morpsJaso_vocab_size, morpsJaso_max_len)
morpsJaso_CNN_model = CNNmodel(morpsJaso_vocab_size, morpsJaso_max_len)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Training

In [9]:
nouns_RNN_history = nouns_RNN_model.fit(nouns_X_train, nouns_y_train, epochs=15, batch_size=60, validation_data=(nouns_X_valid,nouns_y_valid))
print("nouns_RNN 테스트 정확도: %.4f" % (nouns_RNN_model.evaluate(nouns_X_test, nouns_y_test)[1]))

Train on 768 samples, validate on 192 samples
Epoch 1/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.6722 - acc: 0.5911 - val_loss: 0.6394 - val_acc: 0.7604
Epoch 2/15
768/768 [==============================] - 0s 243us/sample - loss: 0.5725 - acc: 0.8737 - val_loss: 0.5795 - val_acc: 0.7604
Epoch 3/15
768/768 [==============================] - 0s 273us/sample - loss: 0.4654 - acc: 0.8971 - val_loss: 0.5270 - val_acc: 0.8177
Epoch 4/15
768/768 [==============================] - 0s 243us/sample - loss: 0.3558 - acc: 0.9193 - val_loss: 0.4598 - val_acc: 0.8333
Epoch 5/15
768/768 [==============================] - 0s 365us/sample - loss: 0.2813 - acc: 0.9271 - val_loss: 0.4108 - val_acc: 0.8438
Epoch 6/15
768/768 [==============================] - 0s 335us/sample - loss: 0.2217 - acc: 0.9440 - val_loss: 0.3922 - val_acc: 0.8438
Epoch 7/15
768/768 [==============================] - 0s 242us/sample - loss: 0.1763 - acc: 0.9609 - val_loss: 0.3990 - val_acc: 0.8333
Epoc

In [10]:
nouns_LSTM_history = nouns_LSTM_model.fit(nouns_X_train, nouns_y_train, epochs=7, batch_size=60, validation_data=(nouns_X_valid,nouns_y_valid))
print("nouns_LSTM 테스트 정확도: %.4f" % (nouns_LSTM_model.evaluate(nouns_X_test, nouns_y_test)[1]))

Train on 768 samples, validate on 192 samples
Epoch 1/7
768/768 [==============================] - 3s 3ms/sample - loss: 1.4563 - acc: 0.4948 - val_loss: 0.7817 - val_acc: 0.5052
Epoch 2/7
768/768 [==============================] - 1s 2ms/sample - loss: 0.7037 - acc: 0.5651 - val_loss: 0.6292 - val_acc: 0.6927
Epoch 3/7
768/768 [==============================] - 1s 2ms/sample - loss: 0.6381 - acc: 0.6328 - val_loss: 0.5708 - val_acc: 0.7708
Epoch 4/7
768/768 [==============================] - 1s 2ms/sample - loss: 0.5471 - acc: 0.7305 - val_loss: 0.5121 - val_acc: 0.7865
Epoch 5/7
768/768 [==============================] - 1s 2ms/sample - loss: 0.4565 - acc: 0.8320 - val_loss: 0.4334 - val_acc: 0.8698
Epoch 6/7
768/768 [==============================] - 1s 2ms/sample - loss: 0.3298 - acc: 0.8945 - val_loss: 0.3580 - val_acc: 0.8802
Epoch 7/7
240/240 [==============================] - 0s 544us/sample - loss: 0.3442 - acc: 0.8708
nouns_LSTM 테스트 정확도: 0.8708


In [11]:
nouns_CNN_history = nouns_CNN_model.fit(nouns_X_train, nouns_y_train, epochs=17, batch_size=60, validation_data=(nouns_X_valid,nouns_y_valid))
print("nouns_CNN 테스트 정확도: %.4f" % (nouns_CNN_model.evaluate(nouns_X_test, nouns_y_test)[1]))


Train on 768 samples, validate on 192 samples
Epoch 1/17
768/768 [==============================] - 1s 887us/sample - loss: 0.6922 - acc: 0.5508 - val_loss: 0.6889 - val_acc: 0.6979
Epoch 2/17
768/768 [==============================] - 0s 122us/sample - loss: 0.6797 - acc: 0.7891 - val_loss: 0.6777 - val_acc: 0.7708
Epoch 3/17
768/768 [==============================] - 0s 119us/sample - loss: 0.6508 - acc: 0.8372 - val_loss: 0.6445 - val_acc: 0.8021
Epoch 4/17
768/768 [==============================] - 0s 114us/sample - loss: 0.5742 - acc: 0.9180 - val_loss: 0.5580 - val_acc: 0.8229
Epoch 5/17
768/768 [==============================] - 0s 162us/sample - loss: 0.4221 - acc: 0.9297 - val_loss: 0.4224 - val_acc: 0.8802
Epoch 6/17
768/768 [==============================] - 0s 119us/sample - loss: 0.2575 - acc: 0.9479 - val_loss: 0.3499 - val_acc: 0.8802
Epoch 7/17
768/768 [==============================] - 0s 117us/sample - loss: 0.1630 - acc: 0.9596 - val_loss: 0.3027 - val_acc: 0.8854
Ep

In [12]:
nounsJaso_RNN_history = nounsJaso_RNN_model.fit(nounsJaso_X_train, nounsJaso_y_train, epochs=15, batch_size=60, validation_data=(nounsJaso_X_valid,nounsJaso_y_valid))
print("nounsJaso_RNN 테스트 정확도: %.4f" % (nounsJaso_RNN_model.evaluate(nounsJaso_X_test, nounsJaso_y_test)[1]))

Train on 768 samples, validate on 192 samples
Epoch 1/15
768/768 [==============================] - 1s 2ms/sample - loss: 0.6613 - acc: 0.6198 - val_loss: 0.6227 - val_acc: 0.6094
Epoch 2/15
768/768 [==============================] - 0s 445us/sample - loss: 0.5204 - acc: 0.7917 - val_loss: 0.5153 - val_acc: 0.7917
Epoch 3/15
768/768 [==============================] - 0s 462us/sample - loss: 0.4098 - acc: 0.8581 - val_loss: 0.4058 - val_acc: 0.8333
Epoch 4/15
768/768 [==============================] - 0s 596us/sample - loss: 0.3571 - acc: 0.8815 - val_loss: 0.3634 - val_acc: 0.8646
Epoch 5/15
768/768 [==============================] - 0s 467us/sample - loss: 0.2918 - acc: 0.9141 - val_loss: 0.3277 - val_acc: 0.8750
Epoch 6/15
768/768 [==============================] - 0s 497us/sample - loss: 0.2354 - acc: 0.9362 - val_loss: 0.5273 - val_acc: 0.6875
Epoch 7/15
768/768 [==============================] - 0s 462us/sample - loss: 0.2324 - acc: 0.9128 - val_loss: 0.3375 - val_acc: 0.8490
Epoc

In [13]:
nounsJaso_LSTM_history = nounsJaso_LSTM_model.fit(nounsJaso_X_train, nounsJaso_y_train, epochs=7, batch_size=60, validation_data=(nounsJaso_X_valid,nounsJaso_y_valid))
print("nounsJaso_LSTM 테스트 정확도: %.4f" % (nounsJaso_LSTM_model.evaluate(nounsJaso_X_test, nounsJaso_y_test)[1]))

Train on 768 samples, validate on 192 samples
Epoch 1/7
768/768 [==============================] - 6s 7ms/sample - loss: 1.4282 - acc: 0.5026 - val_loss: 0.6434 - val_acc: 0.6042
Epoch 2/7
768/768 [==============================] - 4s 5ms/sample - loss: 0.7046 - acc: 0.5182 - val_loss: 0.6338 - val_acc: 0.5052
Epoch 3/7
768/768 [==============================] - 4s 5ms/sample - loss: 0.6090 - acc: 0.6576 - val_loss: 0.5423 - val_acc: 0.7604
Epoch 4/7
768/768 [==============================] - 5s 7ms/sample - loss: 0.5202 - acc: 0.8060 - val_loss: 0.4656 - val_acc: 0.8385
Epoch 5/7
768/768 [==============================] - 4s 5ms/sample - loss: 0.4234 - acc: 0.8542 - val_loss: 0.3625 - val_acc: 0.8698
Epoch 6/7
768/768 [==============================] - 4s 5ms/sample - loss: 0.3141 - acc: 0.9036 - val_loss: 0.3580 - val_acc: 0.8906
Epoch 7/7
240/240 [==============================] - 0s 2ms/sample - loss: 0.3371 - acc: 0.8917
nounsJaso_LSTM 테스트 정확도: 0.8917


In [24]:
nounsJaso_CNN_history = nounsJaso_CNN_model.fit(nounsJaso_X_train, nounsJaso_y_train, epochs=17, batch_size=60, validation_data=(nounsJaso_X_valid,nounsJaso_y_valid))
print("nounsJaso_CNN 테스트 정확도: %.4f" % (nounsJaso_CNN_model.evaluate(nounsJaso_X_test, nounsJaso_y_test)[1]))

Train on 768 samples, validate on 192 samples
Epoch 1/17
768/768 [==============================] - 1s 1ms/sample - loss: 0.6931 - acc: 0.4974 - val_loss: 0.6893 - val_acc: 0.5052
Epoch 2/17
768/768 [==============================] - 0s 178us/sample - loss: 0.6855 - acc: 0.5091 - val_loss: 0.6789 - val_acc: 0.5000
Epoch 3/17
768/768 [==============================] - 0s 209us/sample - loss: 0.6680 - acc: 0.6250 - val_loss: 0.6462 - val_acc: 0.7604
Epoch 4/17
768/768 [==============================] - 0s 200us/sample - loss: 0.6184 - acc: 0.7552 - val_loss: 0.5695 - val_acc: 0.7917
Epoch 5/17
768/768 [==============================] - 0s 177us/sample - loss: 0.5159 - acc: 0.8802 - val_loss: 0.4429 - val_acc: 0.8750
Epoch 6/17
768/768 [==============================] - 0s 191us/sample - loss: 0.3754 - acc: 0.9049 - val_loss: 0.3411 - val_acc: 0.8750
Epoch 7/17
768/768 [==============================] - 0s 208us/sample - loss: 0.2652 - acc: 0.9154 - val_loss: 0.2927 - val_acc: 0.8802
Epoc

In [15]:
morps_RNN_history = morps_RNN_model.fit(morps_X_train, morps_y_train, epochs=15, batch_size=60, validation_data=(morps_X_valid,morps_y_valid))
print("morps_RNN 테스트 정확도: %.4f" % (morps_RNN_model.evaluate(morps_X_test, morps_y_test)[1]))


Train on 768 samples, validate on 192 samples
Epoch 1/15
768/768 [==============================] - 1s 2ms/sample - loss: 0.6776 - acc: 0.5534 - val_loss: 0.6699 - val_acc: 0.5365
Epoch 2/15
768/768 [==============================] - 1s 845us/sample - loss: 0.5968 - acc: 0.7826 - val_loss: 0.6198 - val_acc: 0.7135
Epoch 3/15
768/768 [==============================] - 1s 677us/sample - loss: 0.5029 - acc: 0.8997 - val_loss: 0.5813 - val_acc: 0.7396
Epoch 4/15
768/768 [==============================] - 1s 877us/sample - loss: 0.4070 - acc: 0.9388 - val_loss: 0.5370 - val_acc: 0.7604
Epoch 5/15
768/768 [==============================] - 1s 675us/sample - loss: 0.3330 - acc: 0.9427 - val_loss: 0.5031 - val_acc: 0.7708
Epoch 6/15
768/768 [==============================] - 1s 699us/sample - loss: 0.2503 - acc: 0.9701 - val_loss: 0.4613 - val_acc: 0.7812
Epoch 7/15
768/768 [==============================] - 0s 643us/sample - loss: 0.1858 - acc: 0.9805 - val_loss: 0.4549 - val_acc: 0.8073
Epoc

In [16]:
morps_LSTM_history = morps_LSTM_model.fit(morps_X_train, morps_y_train, epochs=7, batch_size=60, validation_data=(morps_X_valid,morps_y_valid))
print("morps_LSTM 테스트 정확도: %.4f" % (morps_LSTM_model.evaluate(morps_X_test, morps_y_test)[1]))


Train on 768 samples, validate on 192 samples
Epoch 1/7
768/768 [==============================] - 10s 13ms/sample - loss: 1.7651 - acc: 0.4961 - val_loss: 0.7289 - val_acc: 0.5052
Epoch 2/7
768/768 [==============================] - 8s 11ms/sample - loss: 0.7590 - acc: 0.4948 - val_loss: 0.6585 - val_acc: 0.5052
Epoch 3/7
768/768 [==============================] - 8s 11ms/sample - loss: 0.6226 - acc: 0.6628 - val_loss: 0.5910 - val_acc: 0.8229
Epoch 4/7
768/768 [==============================] - 8s 11ms/sample - loss: 0.5457 - acc: 0.8789 - val_loss: 0.5226 - val_acc: 0.8385
Epoch 5/7
768/768 [==============================] - 9s 12ms/sample - loss: 0.4111 - acc: 0.9310 - val_loss: 0.4959 - val_acc: 0.7760
Epoch 6/7
768/768 [==============================] - 8s 11ms/sample - loss: 0.2351 - acc: 0.9453 - val_loss: 0.3829 - val_acc: 0.8021
Epoch 7/7
240/240 [==============================] - 1s 3ms/sample - loss: 0.3141 - acc: 0.8667
morps_LSTM 테스트 정확도: 0.8667


In [17]:
morps_CNN_history = morps_CNN_model.fit(morps_X_train, morps_y_train, epochs=17, batch_size=60, validation_data=(morps_X_valid,morps_y_valid))
print("morps_CNN 테스트 정확도: %.4f" % (morps_CNN_model.evaluate(morps_X_test, morps_y_test)[1]))


Train on 768 samples, validate on 192 samples
Epoch 1/17
768/768 [==============================] - 1s 1ms/sample - loss: 0.6908 - acc: 0.5339 - val_loss: 0.6897 - val_acc: 0.5156
Epoch 2/17
768/768 [==============================] - 0s 365us/sample - loss: 0.6725 - acc: 0.6107 - val_loss: 0.6829 - val_acc: 0.5365
Epoch 3/17
768/768 [==============================] - 0s 416us/sample - loss: 0.6467 - acc: 0.6198 - val_loss: 0.6694 - val_acc: 0.5677
Epoch 4/17
768/768 [==============================] - 0s 265us/sample - loss: 0.5959 - acc: 0.6940 - val_loss: 0.6372 - val_acc: 0.6198
Epoch 5/17
768/768 [==============================] - 0s 279us/sample - loss: 0.5010 - acc: 0.8568 - val_loss: 0.5587 - val_acc: 0.7812
Epoch 6/17
768/768 [==============================] - 0s 278us/sample - loss: 0.3493 - acc: 0.9688 - val_loss: 0.4292 - val_acc: 0.8385
Epoch 7/17
768/768 [==============================] - 0s 287us/sample - loss: 0.1936 - acc: 0.9818 - val_loss: 0.3368 - val_acc: 0.8646
Epoc

In [26]:
morpsJaso_RNN_history = morpsJaso_RNN_model.fit(morpsJaso_X_train, morpsJaso_y_train, epochs=15, batch_size=60, validation_data=(morpsJaso_X_valid,morpsJaso_y_valid))
print("morpsJaso_RNN 테스트 정확도: %.4f" % (morpsJaso_RNN_model.evaluate(morpsJaso_X_test, morpsJaso_y_test)[1]))


Train on 768 samples, validate on 192 samples
Epoch 1/15
768/768 [==============================] - 2s 2ms/sample - loss: 0.6672 - acc: 0.6107 - val_loss: 0.6351 - val_acc: 0.6562
Epoch 2/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.5772 - acc: 0.7995 - val_loss: 0.5857 - val_acc: 0.7396
Epoch 3/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.4818 - acc: 0.8789 - val_loss: 0.5388 - val_acc: 0.7344
Epoch 4/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.4543 - acc: 0.8359 - val_loss: 0.4959 - val_acc: 0.7917
Epoch 5/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.3384 - acc: 0.9362 - val_loss: 0.4493 - val_acc: 0.7865
Epoch 6/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.2699 - acc: 0.9492 - val_loss: 0.4161 - val_acc: 0.7812
Epoch 7/15
768/768 [==============================] - 1s 1ms/sample - loss: 0.2386 - acc: 0.9440 - val_loss: 0.4149 - val_acc: 0.8073
Epoch 8/15
768/7

In [36]:
morpsJaso_LSTM_model = LSTMmodel(morpsJaso_vocab_size, morpsJaso_max_len)


morpsJaso_LSTM_history = morpsJaso_LSTM_model.fit(morpsJaso_X_train, morpsJaso_y_train, epochs=7, batch_size=60, validation_data=(morpsJaso_X_valid,morpsJaso_y_valid))
print("morpsJaso_LSTM 테스트 정확도: %.4f" % (morpsJaso_LSTM_model.evaluate(morpsJaso_X_test, morpsJaso_y_test)[1]))


Train on 768 samples, validate on 192 samples
Epoch 1/7
768/768 [==============================] - 25s 33ms/sample - loss: 1.5876 - acc: 0.4922 - val_loss: 0.9536 - val_acc: 0.5052
Epoch 2/7
768/768 [==============================] - 23s 30ms/sample - loss: 1.0079 - acc: 0.4948 - val_loss: 0.9316 - val_acc: 0.5052
Epoch 3/7
768/768 [==============================] - 24s 32ms/sample - loss: 0.8191 - acc: 0.4948 - val_loss: 0.6668 - val_acc: 0.5052
Epoch 4/7
768/768 [==============================] - 23s 30ms/sample - loss: 0.5961 - acc: 0.6823 - val_loss: 0.5901 - val_acc: 0.7656
Epoch 5/7
768/768 [==============================] - 23s 30ms/sample - loss: 0.5290 - acc: 0.8411 - val_loss: 0.5307 - val_acc: 0.7760
Epoch 6/7
768/768 [==============================] - 27s 35ms/sample - loss: 0.4337 - acc: 0.8438 - val_loss: 0.4717 - val_acc: 0.8125
Epoch 7/7
240/240 [==============================] - 2s 7ms/sample - loss: 0.3474 - acc: 0.8833
morpsJaso_LSTM 테스트 정확도: 0.8833


In [20]:
morpsJaso_CNN_history = morpsJaso_CNN_model.fit(morpsJaso_X_train, morpsJaso_y_train, epochs=17, batch_size=60, validation_data=(morpsJaso_X_valid,morpsJaso_y_valid))
print("morpsJaso_CNN 테스트 정확도: %.4f" % (morpsJaso_CNN_model.evaluate(morpsJaso_X_test, morpsJaso_y_test)[1]))

Train on 768 samples, validate on 192 samples
Epoch 1/17
768/768 [==============================] - 2s 2ms/sample - loss: 0.6911 - acc: 0.5065 - val_loss: 0.6897 - val_acc: 0.5260
Epoch 2/17
768/768 [==============================] - 0s 368us/sample - loss: 0.6749 - acc: 0.6328 - val_loss: 0.6846 - val_acc: 0.5156
Epoch 3/17
768/768 [==============================] - 0s 375us/sample - loss: 0.6547 - acc: 0.6159 - val_loss: 0.6752 - val_acc: 0.5677
Epoch 4/17
768/768 [==============================] - 0s 369us/sample - loss: 0.6208 - acc: 0.7109 - val_loss: 0.6499 - val_acc: 0.5729
Epoch 5/17
768/768 [==============================] - 0s 377us/sample - loss: 0.5680 - acc: 0.7721 - val_loss: 0.5879 - val_acc: 0.7396
Epoch 6/17
768/768 [==============================] - 0s 366us/sample - loss: 0.4513 - acc: 0.9049 - val_loss: 0.4775 - val_acc: 0.8385
Epoch 7/17
768/768 [==============================] - 0s 371us/sample - loss: 0.3020 - acc: 0.9505 - val_loss: 0.3658 - val_acc: 0.8698
Epoc

# Test set Accuracy

In [37]:
#명사 테스트 정확도
print("nouns_RNN 테스트 정확도: %.4f" % (nouns_RNN_model.evaluate(nouns_X_test, nouns_y_test)[1]))
print("nouns_LSTM 테스트 정확도: %.4f" % (nouns_LSTM_model.evaluate(nouns_X_test, nouns_y_test)[1]))
print("nouns_CNN 테스트 정확도: %.4f" % (nouns_CNN_model.evaluate(nouns_X_test, nouns_y_test)[1]))

#명사 자소 분리 테스트 정확도
print("nounsJaso_RNN 테스트 정확도: %.4f" % (nounsJaso_RNN_model.evaluate(nounsJaso_X_test, nounsJaso_y_test)[1]))
print("nounsJaso_LSTM 테스트 정확도: %.4f" % (nounsJaso_LSTM_model.evaluate(nounsJaso_X_test, nounsJaso_y_test)[1]))
print("nounsJaso_CNN 테스트 정확도: %.4f" % (nounsJaso_CNN_model.evaluate(nounsJaso_X_test, nounsJaso_y_test)[1]))

#모든 형태소 정확도
print("morps_RNN 테스트 정확도: %.4f" % (morps_RNN_model.evaluate(morps_X_test, morps_y_test)[1]))
print("morps_LSTM 테스트 정확도: %.4f" % (morps_LSTM_model.evaluate(morps_X_test, morps_y_test)[1]))
print("morps_CNN 테스트 정확도: %.4f" % (morps_CNN_model.evaluate(morps_X_test, morps_y_test)[1]))

#모든 형태소 자소 분리 정확도
print("morpsJaso_RNN 테스트 정확도: %.4f" % (morpsJaso_RNN_model.evaluate(morpsJaso_X_test, morpsJaso_y_test)[1]))
print("morpsJaso_LSTM 테스트 정확도: %.4f" % (morpsJaso_LSTM_model.evaluate(morpsJaso_X_test, morpsJaso_y_test)[1]))
print("morpsJaso_CNN 테스트 정확도: %.4f" % (morpsJaso_CNN_model.evaluate(morpsJaso_X_test, morpsJaso_y_test)[1]))

240/240 [==============================] - 0s 602us/sample - loss: 0.3347 - acc: 0.8542
nouns_RNN 테스트 정확도: 0.8542
240/240 [==============================] - 0s 848us/sample - loss: 0.3442 - acc: 0.8708
nouns_LSTM 테스트 정확도: 0.8708
240/240 [==============================] - 0s 552us/sample - loss: 0.2567 - acc: 0.9042
nouns_CNN 테스트 정확도: 0.9042
240/240 [==============================] - 0s 431us/sample - loss: 0.2735 - acc: 0.8917
nounsJaso_RNN 테스트 정확도: 0.8917
240/240 [==============================] - 0s 2ms/sample - loss: 0.3371 - acc: 0.8917
nounsJaso_LSTM 테스트 정확도: 0.8917
240/240 [==============================] - 0s 399us/sample - loss: 0.2332 - acc: 0.9125
nounsJaso_CNN 테스트 정확도: 0.9125
240/240 [==============================] - 0s 587us/sample - loss: 0.3579 - acc: 0.8458
morps_RNN 테스트 정확도: 0.8458
240/240 [==============================] - 1s 3ms/sample - loss: 0.3141 - acc: 0.8667
morps_LSTM 테스트 정확도: 0.8667
240/240 [==============================] - 0s 387us/sample - loss: 0.2381 - a